In [191]:
import pandas as pd

In [192]:
df = pd.read_csv("../../Downloads/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### checking percentage of missing data

In [193]:
df.isnull().sum()/df.isnull().count()*100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

In [194]:
# Above we can see than about 77% of cabins are not known so it's better to drop this column
df.drop(['Cabin'], axis = 1, inplace = True)
# for age we'll use median imputation
df["Age"].fillna(df["Age"].median(), inplace=True)
#for Embarked we'll fill missing values by mode
df["Embarked"].fillna(df["Embarked"].mode()[0], inplace=True)



In [195]:
for i in df.columns:
    print(i, df[i].nunique())

PassengerId 891
Survived 2
Pclass 3
Name 891
Sex 2
Age 88
SibSp 7
Parch 7
Ticket 681
Fare 248
Embarked 3


In [196]:
#Dropping the name and ID columns as it is categorical with very large nunique()
df.drop(['Name'], axis = 1, inplace= True)
df.drop(['PassengerId'], axis = 1, inplace= True)
#Dropping the Ticket column for same reason
df.drop(['Ticket'], axis = 1, inplace= True)

In [197]:

# Converting all categotical variable into numerical since number of unique entries are large we'll use label encoding and not one_hot
from sklearn.preprocessing import LabelEncoder
lb_enc = LabelEncoder()
s = (df.dtypes == 'object')
object_cols = list(s[s].index)
print(object_cols)
# print(object_cols)
for i in object_cols:
    df[i] = lb_enc.fit_transform(df[i])

df.head()

['Sex', 'Embarked']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


Scaling the age and Fare columns

In [198]:
from sklearn.preprocessing import StandardScaler
std_scl = StandardScaler()
for i in df.columns:
    if(i != "Survived"):
        df[[i]]= std_scl.fit_transform(df[[i]])
# df[['Age']] = std_scl.fit_transform(df[['Age']])
# df[['Fare']] = std_scl.fit_transform(df[['Fare']])
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,0.827377,0.737695,-0.565736,0.432793,-0.473674,-0.502445,0.585954
1,1,-1.566107,-1.355574,0.663861,0.432793,-0.473674,0.786845,-1.942303
2,1,0.827377,-1.355574,-0.258337,-0.474545,-0.473674,-0.488854,0.585954
3,1,-1.566107,-1.355574,0.433312,0.432793,-0.473674,0.420730,0.585954
4,0,0.827377,0.737695,0.433312,-0.474545,-0.473674,-0.486337,0.585954


In [255]:
from sklearn.model_selection import train_test_split
all  = [i for i in df.columns if i != "Survived"]
x = df[all]
y = df['Survived']
x_train, x_val, y_train, y_val= train_test_split(x, y, test_size= 0.25, random_state=5) 

## Training using KNN classifier

In [256]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

def knn_check(neigh):
    knn = KNeighborsClassifier(n_neighbors=neigh)

    knn.fit(x_train,y_train)

    knn_predictions = knn.predict(x_val)
    knn_accuracy = accuracy_score(y_val,knn_predictions)
    print(np.round(knn_accuracy*100,2))

# for i in range(2,50):
#     knn_check(i)
knn_check(25)
    
# 25 seems to be optimal value    

86.1


## Naive Bayes classifier

In [257]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(x_train, y_train)
nb_predictions = nb.predict(x_val)
nb_accuracy = accuracy_score(y_val,nb_predictions)
print(np.round(nb_accuracy*100,2))


83.86


## Decision Tree Classifier 

In [258]:
from sklearn.tree import DecisionTreeClassifier

def dt_acc(depth):
    
    dt = DecisionTreeClassifier(random_state=0, max_depth= depth )
    dt.fit(x_train, y_train)
    dt_predictions =dt.predict(x_val)
    dt_accuraccy = accuracy_score(y_val, dt_predictions)
    print(np.round(dt_accuraccy*100,2))
    
# for i in range(1,20):                 
#     dt_acc(i)
dt_acc(7)
#best accuraccy obtained for max_depth = 6     

79.82


In [259]:
from xgboost import XGBClassifier

def xgb_acc(lr, n_estim):
    xgb = XGBClassifier(eta = lr , n_estimators = n_estim,early_stopping_rounds=5)
    xgb.fit(x_train, y_train, eval_set=[(x_val, y_val),(x_train, y_train)], verbose= False)
    # xgb_predictions = xgb.predict(x_val)
    xgb_accuraccy = xgb.score(x_val,y_val)
    xgb_test_score = xgb.score(x_train,y_train)
    # results = xgb.evals_result()
    # epochs = len(results["validation_0"]["accuracy"])
    # print(lr, n_estim)
    # print(results["validation_0"]["accuracy"][epochs - 1])
    print("test_score", np.round(xgb_accuraccy*100,2))
    # print("train_score", np.round(xgb_test_score*100,2), "\n")
# for i in np.arange(0.01, 0.03, 0.005):
#     for j in range(100,800,100):
#         xgb_acc(i,j)  
xgb_acc(0.02,200) 

test_score 83.86
